# Weights

## Load Graph

In [1]:
import os
import sys
from pathlib import Path
import plotly.graph_objects as go
import plotly.io as pio
from dotenv import load_dotenv

# Add the src directory to the Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.maritime_module.utils.port_utils import Boundaries
from src.maritime_module.utils.plot_utils import PlotlyChart
from src.maritime_module.utils.geometry_utils import Buffer
import geopandas as gpd
from shapely.geometry import shape, LineString, MultiPolygon, Point, Polygon, box


from src.maritime_module.core.graph import H3Graph, Weights
from src.maritime_module.core.s57_data import ENCDataFactory

# Load environment variables from .env file at the project root
load_dotenv(project_root / ".env")
pio.renderers.default = "notebook_connected"

output_dir = Path.cwd() / 'output'
output_dir.mkdir(exist_ok=True)

db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

factory = ENCDataFactory(source=db_params, schema="us_enc_all")
h3 = H3Graph(data_factory=factory,
             route_schema_name="routes",
             graph_schema_name="graph")

2025-10-09 13:06:04,447 - src.maritime_module.core.s57_data - INFO - Source is a dictionary, initializing PostGISManager.
2025-10-09 13:06:04,478 - src.maritime_module.core.s57_data - INFO - Successfully connected to database 'ENC_db'


In [2]:
from src.maritime_module.utils.db_utils import PostGISConnector

pg = PostGISConnector(db_params=db_params, schema="graph")
# pg.drop_columns(schema_name="graph",
#                 table_name="fine_graph_01_opt_edges",
#                 columns=["final_weight", "static_weight_factor"])

In [3]:
graph_name = "h3_graph_PG_6_11"
# Initialize the Weights manager with the same data factory
weights_manager = Weights(data_factory=factory)
weights_manager.clean_graph_postgis(graph_name)

2025-10-09 13:06:04,587 - src.maritime_module.core.graph - INFO - Weights manager initialized with default S57 classifier
2025-10-09 13:06:04,588 - src.maritime_module.core.graph - INFO - Default static layers from config: 9 layers
2025-10-09 13:06:04,588 - src.maritime_module.core.graph - INFO - === Cleaning PostGIS Graph ===
2025-10-09 13:06:04,589 - src.maritime_module.core.graph - INFO - Table: graph.h3_graph_PG_6_11_edges
2025-10-09 13:06:04,646 - src.maritime_module.core.graph - INFO - Dropping 19 columns...
2025-10-09 13:06:04,708 - src.maritime_module.core.graph - INFO - === PostGIS Graph Cleaned ===
2025-10-09 13:06:04,709 - src.maritime_module.core.graph - INFO - Dropped 19 columns
2025-10-09 13:06:04,709 - src.maritime_module.core.graph - INFO - Kept 9 columns (including weight, geom)


{'columns_dropped': 19,
 'columns_kept': ['id',
  'source_str',
  'target_str',
  'source_x',
  'source_y',
  'target_x',
  'target_y',
  'weight',
  'geometry'],
 'columns_removed': ['base_weight',
  'adjusted_weight',
  'blocking_factor',
  'penalty_factor',
  'bonus_factor',
  'ukc_meters',
  'ft_depth_sources',
  'ft_hor_clearance',
  'ft_hor_clearance_sources',
  'ft_ver_clearance',
  'ft_ver_clearance_sources',
  'ft_depth',
  'ft_sounding',
  'ft_sounding_sources',
  'ft_sounding_point',
  'ft_sounding_point_sources',
  'wt_static_blocking',
  'wt_static_penalty',
  'wt_static_bonus']}

In [4]:
# Load a pre-existing graph from PostGIS
# This assumes a graph was previously saved with a prefix like 'fine_graph_01'

try:
    G = h3.load_graph_from_postgis(table_prefix=graph_name)
    print(f"Successfully loaded graph with {G.number_of_nodes():,} nodes and {G.number_of_edges():,} edges.")
except Exception as e:
    print(f"Could not load graph. Please ensure a graph has been saved to PostGIS first. Error: {e}")
    G = None

# 1. Get the boundary of the graph itself
nodes_df = gpd.GeoDataFrame(geometry=[Point(n) for n in G.nodes()], crs="EPSG:4326")
graph_boundary = nodes_df.union_all().convex_hull

2025-10-09 13:06:04,744 - src.maritime_module.core.graph - INFO - Loading graph from PostGIS schema 'graph' tables: h3_graph_PG_6_11_nodes, h3_graph_PG_6_11_edges
2025-10-09 13:06:51,076 - src.maritime_module.core.graph - INFO - Loaded and processed 931,149 nodes in 46.330s
2025-10-09 13:11:25,104 - src.maritime_module.core.graph - INFO - Loaded and processed 2,784,887 edges in 274.027s
2025-10-09 13:11:26,109 - src.maritime_module.core.graph - INFO - Graph loaded from PostGIS: 931,149 nodes, 2,784,887 edges in 320.863s
2025-10-09 13:11:26,110 - src.maritime_module.core.graph - INFO - === PostGIS Graph Load Operation Performance Summary ===
2025-10-09 13:11:26,111 - src.maritime_module.core.graph - INFO - Timing Metrics:
2025-10-09 13:11:26,112 - src.maritime_module.core.graph - INFO -   nodes_load_time: 7.289s
2025-10-09 13:11:26,112 - src.maritime_module.core.graph - INFO -   nodes_processing_time: 39.041s
2025-10-09 13:11:26,112 - src.maritime_module.core.graph - INFO -   edges_load

 ## Initialize Weights Class

 ## Apply Static Feature Weights

In [5]:
# 2. Get the list of ENCs intersecting this boundary
enc_list = factory.get_encs_by_boundary(graph_boundary)
print(f"Found {len(enc_list)} ENCs intersecting with the graph boundary.")

ft_weights = weights_manager.get_feature_layers_from_classifier()

ft_weights


2025-10-09 13:11:32,014 - src.maritime_module.core.s57_data - INFO - Factory: Filtering ENCs by boundary...
2025-10-09 13:11:32,015 - src.maritime_module.core.s57_data - INFO - Factory: Getting ENC summary...
2025-10-09 13:11:32,034 - src.maritime_module.core.s57_data - INFO - Factory: Getting bounding boxes for 6369 ENCs...
Found 19 ENCs intersecting with the graph boundary.
2025-10-09 13:11:32,409 - src.maritime_module.core.graph - INFO - Generated 21 feature layer configs from classifier


{'canals': {'column': 'ft_hor_clearance',
  'attributes': ['horclr'],
  'aggregation': 'min'},
 'convyr': {'column': 'ft_ver_clearance',
  'attributes': ['verclr'],
  'aggregation': 'min'},
 'depare': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'swpare': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'drgare': {'column': 'ft_depth',
  'attributes': ['drval1'],
  'aggregation': 'min'},
 'docare': {'column': 'ft_hor_clearance',
  'attributes': ['horclr'],
  'aggregation': 'min'},
 'marcul': {'column': 'ft_sounding',
  'attributes': ['valsou'],
  'aggregation': 'min'},
 'cblohd': {'column': 'ft_ver_clearance',
  'attributes': ['verclr', 'vercsa'],
  'aggregation': 'min'},
 'pipohd': {'column': 'ft_ver_clearance',
  'attributes': ['verclr'],
  'aggregation': 'min'},
 'flodoc': {'column': 'ft_hor_clearance',
  'attributes': ['horclr'],
  'aggregation': 'min'},
 'cranes': {'column': 'ft_ver_clearance',
  'attributes': ['verclr'],

In [6]:
# 3. Apply weights from static layers (e.g., land, fairways, obstructions)
summary = weights_manager.enrich_edges_with_features_postgis(enc_names= enc_list,
                                                               schema_name="graph",
                                                               graph_name=graph_name,
                                                               enc_schema="us_enc_all",
                                                               feature_layers=ft_weights
                                                             )



summary

2025-10-09 13:11:32,419 - src.maritime_module.core.graph - INFO - Generated 21 feature layer configs from classifier
2025-10-09 13:11:32,420 - src.maritime_module.core.graph - INFO - === PostGIS Feature Enrichment (Server-Side) ===
2025-10-09 13:11:32,421 - src.maritime_module.core.graph - INFO - Edges table: graph.h3_graph_PG_6_11_edges
2025-10-09 13:11:32,421 - src.maritime_module.core.graph - INFO - Layers schema: us_enc_all
2025-10-09 13:11:32,422 - src.maritime_module.core.graph - INFO - Processing 21 feature layers
2025-10-09 13:11:32,422 - src.maritime_module.core.graph - INFO - Initializing weight calculation columns
2025-10-09 13:11:32,428 - src.maritime_module.core.graph - INFO - Added column 'base_weight' to h3_graph_PG_6_11_edges
2025-10-09 13:11:32,434 - src.maritime_module.core.graph - INFO - Added column 'adjusted_weight' to h3_graph_PG_6_11_edges
2025-10-09 13:11:32,440 - src.maritime_module.core.graph - INFO - Added column 'blocking_factor' to h3_graph_PG_6_11_edges
20

{'ft_hor_clearance': 0,
 'ft_ver_clearance': 2119,
 'ft_depth': 2813580,
 'ft_sounding': 782,
 'ft_sounding_point': 0}

In [7]:
pg.get_table(table_name=f"{graph_name}_edges",
             schema_name="graph")

2025-10-09 13:19:08,176 - src.maritime_module.utils.db_utils - INFO - Successfully connected to database 'ENC_db' for schema management.
2025-10-09 13:20:29,019 - src.maritime_module.utils.db_utils - INFO - Loaded 2784887 rows from 'graph.h3_graph_PG_6_11_edges'


,id,source_str,target_str,source_x,source_y,target_x,target_y,weight,geometry,base_weight,...,ft_depth_sources,ft_hor_clearance,ft_hor_clearance_sources,ft_ver_clearance,ft_ver_clearance_sources,ft_depth,ft_sounding,ft_sounding_sources,ft_sounding_point,ft_sounding_point_sources
0,36653,"(-122.70442528145516, 37.29290314155771)","(-122.70105009172504, 37.29477183788321)",-122.704425,37.292903,-122.701050,37.294772,0.196414,"LINESTRING (-122.70443 37.2929, -122.70105 37....",0.196414,...,"{'US1WC01M_depare': {'value': 30, 'usage_band'...",None,None,NaN,None,91.4,NaN,None,None,None
1,2558810,"(-122.34777821998698, 37.72649185682434)","(-122.34826442428414, 37.72622623378993)",-122.347778,37.726492,-122.348264,37.726226,0.028062,"LINESTRING (-122.34778 37.72649, -122.34826 37...",0.028062,...,"{'US1WC01M_depare': {'value': 0, 'usage_band':...",None,None,NaN,None,18.2,NaN,None,None,None
2,2440968,"(-122.62412206689525, 37.76971930322309)","(-122.62414982824414, 37.77017429088422)",-122.624122,37.769719,-122.624150,37.770174,0.027349,"LINESTRING (-122.62412 37.76972, -122.62415 37...",0.027349,...,"{'US1WC01M_depare': {'value': 0, 'usage_band':...",None,None,NaN,None,15.4,NaN,None,None,None
3,1350547,"(-122.33270390719495, 37.760578187631054)","(-122.33321689246519, 37.760768150009845)",-122.332704,37.760578,-122.333217,37.760768,0.026888,"LINESTRING (-122.3327 37.76058, -122.33322 37....",0.026888,...,"{'US1WC01M_depare': {'value': 0, 'usage_band':...",None,None,NaN,None,9.1,NaN,None,None,None
4,2456818,"(-122.28730231963732, 37.79145427435659)","(-122.28681546396929, 37.79171966556907)",-122.287302,37.791454,-122.286815,37.791720,0.028062,"LINESTRING (-122.2873 37.79145, -122.28682 37....",0.028062,...,"{'US5CA13M_drgare': {'value': 14.5, 'usage_ban...",None,None,NaN,None,14.5,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2784882,2223941,"(-122.6538318318921, 37.559457571019436)","(-122.65434334046668, 37.55964657678767)",-122.653832,37.559458,-122.654343,37.559647,0.026860,"LINESTRING (-122.65383 37.55946, -122.65434 37...",0.026860,...,"{'US1WC01M_depare': {'value': 30, 'usage_band'...",None,None,NaN,None,36.5,NaN,None,None,None
2784883,325045,"(-122.44243385016007, 37.90171951332442)","(-122.44194736091958, 37.90198535289974)",-122.442434,37.901720,-122.441947,37.901985,0.028035,"LINESTRING (-122.44243 37.90172, -122.44195 37...",0.028035,...,"{'US1WC01M_depare': {'value': 0, 'usage_band':...",None,None,NaN,None,9.1,NaN,None,None,None
2784884,2328169,"(-122.7904630686994, 37.9305951305571)","(-122.7899784467906, 37.930862081351414)",-122.790463,37.930595,-122.789978,37.930862,0.027993,"LINESTRING (-122.79046 37.9306, -122.78998 37....",0.027993,...,"{'US1WC01M_depare': {'value': 30, 'usage_band'...",None,None,NaN,None,18.2,NaN,None,None,None
2784885,1366157,"(-122.88425853358922, 37.87650408249214)","(-122.88474235412144, 37.87623680277327)",-122.884259,37.876504,-122.884742,37.876237,0.027987,"LINESTRING (-122.88426 37.8765, -122.88474 37....",0.027987,...,"{'US1WC01M_depare': {'value': 30, 'usage_band'...",None,None,NaN,None,54.8,NaN,None,None,None


In [8]:
config = weights_manager._load_config()

In [9]:
weights_manager.apply_static_weights_postgis(graph_name=graph_name,
                                             enc_names=enc_list,
                                             schema_name="graph",
                                             enc_schema="us_enc_all",
                                             static_layers=config["weight_settings"]["static_layers"],
                                             usage_bands=[3,4,5])

print("Static weights applied.")

2025-10-09 13:20:30,945 - src.maritime_module.core.graph - INFO - Filtered 19 ENCs to 16 based on usage bands [3, 4, 5]
2025-10-09 13:20:30,946 - src.maritime_module.core.graph - INFO - === PostGIS Static Weights Application (Three-Tier System) ===
2025-10-09 13:20:30,946 - src.maritime_module.core.graph - INFO - Edges table: graph.h3_graph_PG_6_11_edges
2025-10-09 13:20:30,947 - src.maritime_module.core.graph - INFO - Layers schema: us_enc_all
2025-10-09 13:20:30,947 - src.maritime_module.core.graph - INFO - Processing 9 layers
2025-10-09 13:20:30,947 - src.maritime_module.core.graph - INFO - Ensuring three-tier weight columns exist...
2025-10-09 13:20:31,953 - src.maritime_module.core.graph - INFO - Added 'wt_static_blocking' column to h3_graph_PG_6_11_edges
2025-10-09 13:20:31,955 - src.maritime_module.core.graph - INFO - Added 'wt_static_penalty' column to h3_graph_PG_6_11_edges
2025-10-09 13:20:31,957 - src.maritime_module.core.graph - INFO - Added 'wt_static_bonus' column to h3_g

In [10]:
pg.get_table(table_name=f"{graph_name}_edges",
             schema_name="graph")

2025-10-09 13:24:32,947 - src.maritime_module.utils.db_utils - INFO - Loaded 2784887 rows from 'graph.h3_graph_PG_6_11_edges'


,id,source_str,target_str,source_x,source_y,target_x,target_y,weight,geometry,base_weight,...,ft_ver_clearance,ft_ver_clearance_sources,ft_depth,ft_sounding,ft_sounding_sources,ft_sounding_point,ft_sounding_point_sources,wt_static_blocking,wt_static_penalty,wt_static_bonus
0,36653,"(-122.70442528145516, 37.29290314155771)","(-122.70105009172504, 37.29477183788321)",-122.704425,37.292903,-122.701050,37.294772,0.196414,"LINESTRING (-122.70443 37.2929, -122.70105 37....",0.196414,...,NaN,None,91.4,NaN,None,None,None,1.0,1.0,1.00
1,2558810,"(-122.34777821998698, 37.72649185682434)","(-122.34826442428414, 37.72622623378993)",-122.347778,37.726492,-122.348264,37.726226,0.028062,"LINESTRING (-122.34778 37.72649, -122.34826 37...",0.028062,...,NaN,None,18.2,NaN,None,None,None,1.0,1.0,1.00
2,2440968,"(-122.62412206689525, 37.76971930322309)","(-122.62414982824414, 37.77017429088422)",-122.624122,37.769719,-122.624150,37.770174,0.027349,"LINESTRING (-122.62412 37.76972, -122.62415 37...",0.027349,...,NaN,None,15.4,NaN,None,None,None,5.0,1.0,0.45
3,2456818,"(-122.28730231963732, 37.79145427435659)","(-122.28681546396929, 37.79171966556907)",-122.287302,37.791454,-122.286815,37.791720,0.028062,"LINESTRING (-122.2873 37.79145, -122.28682 37....",0.028062,...,NaN,None,14.5,NaN,None,None,None,180.0,1.0,0.45
4,1350547,"(-122.33270390719495, 37.760578187631054)","(-122.33321689246519, 37.760768150009845)",-122.332704,37.760578,-122.333217,37.760768,0.026888,"LINESTRING (-122.3327 37.76058, -122.33322 37....",0.026888,...,NaN,None,9.1,NaN,None,None,None,1.0,1.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2784882,2223941,"(-122.6538318318921, 37.559457571019436)","(-122.65434334046668, 37.55964657678767)",-122.653832,37.559458,-122.654343,37.559647,0.026860,"LINESTRING (-122.65383 37.55946, -122.65434 37...",0.026860,...,NaN,None,36.5,NaN,None,None,None,5.0,1.0,1.00
2784883,2328169,"(-122.7904630686994, 37.9305951305571)","(-122.7899784467906, 37.930862081351414)",-122.790463,37.930595,-122.789978,37.930862,0.027993,"LINESTRING (-122.79046 37.9306, -122.78998 37....",0.027993,...,NaN,None,18.2,NaN,None,None,None,1.0,1.0,1.00
2784884,325045,"(-122.44243385016007, 37.90171951332442)","(-122.44194736091958, 37.90198535289974)",-122.442434,37.901720,-122.441947,37.901985,0.028035,"LINESTRING (-122.44243 37.90172, -122.44195 37...",0.028035,...,NaN,None,9.1,NaN,None,None,None,1.0,1.0,0.50
2784885,1366157,"(-122.88425853358922, 37.87650408249214)","(-122.88474235412144, 37.87623680277327)",-122.884259,37.876504,-122.884742,37.876237,0.027987,"LINESTRING (-122.88426 37.8765, -122.88474 37....",0.027987,...,NaN,None,54.8,NaN,None,None,None,1.0,1.0,1.00


In [11]:
ply = PlotlyChart()
ply_fig = ply.create_base_map(mapbox_token="pk.eyJ1Ijoidmlrb250IiwiYSI6ImNtNHljdzQwbjBybWIybHNkZzhleXhqYXUifQ.Jz0cR4J6PdAnLutVBqj2WA")
ply.plotly_base_config(ply_fig)

{'displayModeBar': 'hover',
 'responsive': True,
 'scrollZoom': True,
 'displaylogo': False,
 'modeBarButtonsToRemove': ['zoomIn',
  'zoomOut',
  'pan',
  'select',
  'lassoSelect'],
 'modeBarButtonsToAdd': ['autoScale', 'hoverCompareCartesian']}

 ## Calculate Dynamic Weights

In [12]:

# Define vessel parameters for dynamic weight calculation
vessel_params = {
 'draft': 7.5,           # meters
 'height': 30.0,         # meters (for overhead clearance)
 'safety_margin': 2.0,   # meters (for under-keel clearance)
 'vessel_type': 'cargo'
}

# Define environmental conditions (optional)
env_conditions = {
 'weather_factor': 1.2,      # 1.0=good, >1.0=poor
 'visibility_factor': 1.1,   # 1.0=good, >1.0=poor
 'time_of_day': 'day'        # 'day' or 'night'
}

# Calculate the final weights
# This function uses the three-tier system: Blocking, Penalties, and Bonuses
weights_manager.calculate_dynamic_weights_postgis(
 graph_name=graph_name,
 schema_name="graph",
 vessel_parameters=vessel_params,
 environmental_conditions=env_conditions,
)

print("Dynamic weights calculated.")

2025-10-09 13:24:34,442 - src.maritime_module.core.graph - INFO - === Dynamic Weight Calculation (PostGIS - Three-Tier System) ===
2025-10-09 13:24:34,443 - src.maritime_module.core.graph - INFO - Vessel: type=cargo, draft=7.5m, height=30.0m
2025-10-09 13:24:34,444 - src.maritime_module.core.graph - INFO - Safety margin: 2.0m → 2.64m (adjusted)
2025-10-09 13:24:34,444 - src.maritime_module.core.graph - INFO - Environment: weather=1.2, visibility=1.1, time=day
2025-10-09 13:24:34,444 - src.maritime_module.core.graph - INFO - Max penalty cap: 50.0
2025-10-09 13:26:14,806 - src.maritime_module.core.graph - INFO - Tier 1: Calculating blocking factors...
2025-10-09 13:27:09,783 - src.maritime_module.core.graph - INFO - Tier 2: Calculating penalty factors...
2025-10-09 13:27:44,792 - src.maritime_module.core.graph - INFO - Tier 3: Calculating bonus factors...
2025-10-09 13:28:43,339 - src.maritime_module.core.graph - INFO - Calculating adjusted weights...
2025-10-09 13:30:16,578 - src.mariti

## Inspect Weighted Graph

In [13]:
pg.get_table(table_name=f"{graph_name}_edges",
             schema_name="graph")

2025-10-09 13:31:41,501 - src.maritime_module.utils.db_utils - INFO - Loaded 2784887 rows from 'graph.h3_graph_PG_6_11_edges'


,id,source_str,target_str,source_x,source_y,target_x,target_y,weight,geometry,base_weight,...,ft_ver_clearance,ft_ver_clearance_sources,ft_depth,ft_sounding,ft_sounding_sources,ft_sounding_point,ft_sounding_point_sources,wt_static_blocking,wt_static_penalty,wt_static_bonus
0,36653,"(-122.70442528145516, 37.29290314155771)","(-122.70105009172504, 37.29477183788321)",-122.704425,37.292903,-122.701050,37.294772,0.196414,"LINESTRING (-122.70443 37.2929, -122.70105 37....",0.196414,...,NaN,None,91.4,NaN,None,None,None,1.0,1.0,1.00
1,2558810,"(-122.34777821998698, 37.72649185682434)","(-122.34826442428414, 37.72622623378993)",-122.347778,37.726492,-122.348264,37.726226,0.028062,"LINESTRING (-122.34778 37.72649, -122.34826 37...",0.028062,...,NaN,None,18.2,NaN,None,None,None,1.0,1.0,1.00
2,2456818,"(-122.28730231963732, 37.79145427435659)","(-122.28681546396929, 37.79171966556907)",-122.287302,37.791454,-122.286815,37.791720,0.028062,"LINESTRING (-122.2873 37.79145, -122.28682 37....",0.028062,...,NaN,None,14.5,NaN,None,None,None,180.0,1.0,0.45
3,2440968,"(-122.62412206689525, 37.76971930322309)","(-122.62414982824414, 37.77017429088422)",-122.624122,37.769719,-122.624150,37.770174,0.027349,"LINESTRING (-122.62412 37.76972, -122.62415 37...",0.027349,...,NaN,None,15.4,NaN,None,None,None,5.0,1.0,0.45
4,1350547,"(-122.33270390719495, 37.760578187631054)","(-122.33321689246519, 37.760768150009845)",-122.332704,37.760578,-122.333217,37.760768,0.026888,"LINESTRING (-122.3327 37.76058, -122.33322 37....",0.026888,...,NaN,None,9.1,NaN,None,None,None,1.0,1.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2784882,2223941,"(-122.6538318318921, 37.559457571019436)","(-122.65434334046668, 37.55964657678767)",-122.653832,37.559458,-122.654343,37.559647,0.026860,"LINESTRING (-122.65383 37.55946, -122.65434 37...",0.026860,...,NaN,None,36.5,NaN,None,None,None,5.0,1.0,1.00
2784883,2328169,"(-122.7904630686994, 37.9305951305571)","(-122.7899784467906, 37.930862081351414)",-122.790463,37.930595,-122.789978,37.930862,0.027993,"LINESTRING (-122.79046 37.9306, -122.78998 37....",0.027993,...,NaN,None,18.2,NaN,None,None,None,1.0,1.0,1.00
2784884,325045,"(-122.44243385016007, 37.90171951332442)","(-122.44194736091958, 37.90198535289974)",-122.442434,37.901720,-122.441947,37.901985,0.028035,"LINESTRING (-122.44243 37.90172, -122.44195 37...",0.028035,...,NaN,None,9.1,NaN,None,None,None,1.0,1.0,0.50
2784885,1366157,"(-122.88425853358922, 37.87650408249214)","(-122.88474235412144, 37.87623680277327)",-122.884259,37.876504,-122.884742,37.876237,0.027987,"LINESTRING (-122.88426 37.8765, -122.88474 37....",0.027987,...,NaN,None,54.8,NaN,None,None,None,1.0,1.0,1.00


 ## Save the Weighted Graph

In [14]:
output_path = output_dir / "weighted_graph_new5.gpkg"
h3.export_postgis_to_gpkg(schema_name="graph",
                          graph_name=graph_name,
                            output_path=str(output_path))


2025-10-09 13:31:42,928 - src.maritime_module.core.graph - INFO - === Exporting PostGIS to GeoPackage ===
2025-10-09 13:31:42,928 - src.maritime_module.core.graph - INFO - Source: graph.h3_graph_PG_6_11_nodes, graph.h3_graph_PG_6_11_edges
2025-10-09 13:31:42,928 - src.maritime_module.core.graph - INFO - Target: /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output/weighted_graph_new5.gpkg
2025-10-09 13:31:42,929 - src.maritime_module.core.graph - INFO - Exporting nodes table...
2025-10-09 13:31:49,134 - src.maritime_module.core.graph - INFO - Nodes exported in 6.205s
2025-10-09 13:31:49,135 - src.maritime_module.core.graph - INFO - Exporting edges table...
2025-10-09 13:32:36,299 - src.maritime_module.core.graph - INFO - Edges exported in 47.164s
2025-10-09 13:32:37,493 - src.maritime_module.core.graph - INFO - === Export Complete ===
2025-10-09 13:32:37,495 - src.maritime_module.core.graph - INFO - Exported 931,149 nodes, 2,784,887 edges
2025-10-09 13:32:37,4

{'node_count': 931149,
 'edge_count': 2784887,
 'output_path': '/home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output/weighted_graph_new5.gpkg',
 'total_time': 54.56614867999997}

In [15]:
# import geopandas as gpd
# from typing import List
# layer_name = "drgare"
# layer = factory.get_layer(layer_name=layer_name,
#                           filter_by_enc=enc_list)
# layer_path = output_dir / f"layer_{layer_name}.gpkg"
# layer.to_file(layer_path, driver="GPKG", engine="fiona")

In [16]:
# def save_layer_by_usage_band(factory, layer_name: str, enc_list: List[str], output_path: str):
# 		"""
# 		Fetches a layer and saves it to a GeoPackage, with each usage band as a separate layer.
#
# 		This is useful for inspecting how different navigational scale data (e.g., overview vs. harbour)
# 		is represented for a given S-57 feature.
#
# 		Args:
# 			factory: An initialized ENCDataFactory instance.
# 			layer_name (str): The name of the S-57 layer to process (e.g., 'depare', 'obstrn').
# 			enc_list (List[str]): A list of ENC names to filter the data.
# 			output_path (str): The path for the output GeoPackage file.
#
# 		Example:
# 			ply = PlotlyChart()
# 			output_file = output_dir / "depare_by_band.gpkg"
# 			ply.save_layer_by_usage_band(
# 				factory=pg_factory,
# 				layer_name="depare",
# 				enc_list=enc_list,
# 				output_path=str(output_file)
# 			)
# 		"""
# 		print(f"--- Saving layer '{layer_name}' to '{output_path}' by usage band ---")
#
# 		# Fetch the entire layer for the given ENCs first to avoid multiple DB calls
# 		try:
# 			full_layer_gdf = factory.get_layer(layer_name=layer_name, filter_by_enc=enc_list)
# 			if full_layer_gdf.empty:
# 				print(f"Layer '{layer_name}' is empty or not found for the given ENCs. No file created.")
# 				return
# 		except Exception as e:
# 			print(f"Error fetching layer '{layer_name}': {e}")
# 			return
#
# 		# Iterate through usage bands 1 to 6
# 		for i in range(1, 7):
# 			band_encs = [enc for enc in enc_list if len(enc) > 2 and enc[2] == str(i)]
# 			if not band_encs:
# 				continue
#
# 			# Filter the GeoDataFrame in memory
# 			band_gdf = full_layer_gdf[full_layer_gdf['dsid_dsnm'].isin(band_encs)]
#
# 			if not band_gdf.empty:
# 				band_layer_name = f"{layer_name}_band_{i}"
# 				# Use mode 'a' (append) for subsequent layers
# 				mode = 'w' if i == 1 else 'a'
# 				band_gdf.to_file(output_path, layer=band_layer_name, driver="GPKG", mode=mode)
# 				print(f"Saved {len(band_gdf)} features to layer '{band_layer_name}'")

In [17]:
# layer_path = output_dir / f"layer_{layer_name}_multi.gpkg"
# save_layer_by_usage_band(factory=factory,
#                          layer_name="depare",
#                          enc_list=enc_list,
#                          output_path=str(layer_path))